In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import numpy as np
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# تنظیمات
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 5
batch_size = 64
num_epochs = 100
output_dir = "F:\\payan-nameh\\faz2 . 1404.04.02\\Date\\RNALocate\\"

# 🔹 بارگذاری داده‌ها
X_train_attention = np.load(f"{output_dir}X_train_attention.npy")
X_val_attention = np.load(f"{output_dir}X_val_attention.npy")
X_test_attention = np.load(f"{output_dir}X_test_attention.npy")
y_train = np.load(f"{output_dir}y_train.npy")
y_val = np.load(f"{output_dir}y_val.npy")
y_test = np.load(f"{output_dir}y_test.npy")
X_gan = np.load(f"{output_dir}X_train_augmented.npy")
y_gan = np.load(f"{output_dir}y_train_augmented.npy")

# هماهنگ‌سازی ابعاد GAN با شبکه
class GanReducer(nn.Module):
    def __init__(self, input_dim=512, output_dim=256):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )
    def forward(self, x):
        return self.linear(x)

gan_reducer = GanReducer(input_dim=X_gan.shape[1], output_dim=X_train_attention.shape[1]).to(device)
X_gan = X_gan.astype(np.float32)
X_gan = torch.FloatTensor(X_gan).to(device)
with torch.no_grad():
    X_gan = gan_reducer(X_gan).cpu().numpy()

# ترکیب و شافل
X_train_combined = np.concatenate([X_train_attention, X_gan], axis=0)
y_train_combined = np.concatenate([y_train, y_gan], axis=0)
X_train_combined, y_train_combined = shuffle(X_train_combined, y_train_combined, random_state=42)
input_dim = X_train_combined.shape[1]

# ساخت دیتاست و لودر
train_dataset = TensorDataset(torch.FloatTensor(X_train_combined), torch.LongTensor(y_train_combined))
val_dataset = TensorDataset(torch.FloatTensor(X_val_attention), torch.LongTensor(y_val))
test_dataset = TensorDataset(torch.FloatTensor(X_test_attention), torch.LongTensor(y_test))

# استفاده از WeightedRandomSampler (راه حل 2)
class_counts = np.bincount(y_train_combined)
weights = 1.0 / class_counts[y_train_combined]
sampler = WeightedRandomSampler(weights, len(y_train_combined), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 🔶 گام 5.3: ساخت مدل نهایی
class FinalClassifier(nn.Module):
    def __init__(self, input_dim, num_classes=5):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),  # معماری عمیق‌تر (راه حل 3)
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.model(x)

model = FinalClassifier(input_dim=input_dim).to(device)

# 🔶 گام 5.4: تعریف loss + optimizer + scheduler
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_combined), y=y_train_combined)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)  # کاهش lr (راه حل 4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)  # افزایش patience (راه حل 1)

# 🔶 گام 5.5: پیاده‌سازی Early Stopping
best_val_f1 = 0.0
patience_counter = 0
val_f1_list, val_acc_list = [], []

for epoch in range(num_epochs):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = loss_fn(out, yb)
        loss.backward()
        optimizer.step()

    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_preds.extend(out.argmax(dim=1).cpu().numpy())
            val_labels.extend(yb.cpu().numpy())

    val_f1 = f1_score(val_labels, val_preds, average='macro')
    val_acc = accuracy_score(val_labels, val_preds)
    val_f1_list.append(val_f1)
    val_acc_list.append(val_acc)
    print(f"Epoch {epoch+1}: Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), f"{output_dir}best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= 10:  # افزایش patience
            print("Early stopping triggered!")
            break

    scheduler.step(1.0 - val_f1)

# 🟩 مرحله 6: ارزیابی و تحلیل نتایج
model.load_state_dict(torch.load(f"{output_dir}best_model.pth"))
model.eval()
all_preds, all_probs, all_labels = [], [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        all_preds.extend(out.argmax(dim=1).cpu().numpy())
        all_probs.extend(out.cpu().numpy())
        all_labels.extend(yb.cpu().numpy())

np.save(f"{output_dir}all_preds.npy", np.array(all_preds))
np.save(f"{output_dir}all_probs.npy", np.array(all_probs))
np.save(f"{output_dir}all_labels.npy", np.array(all_labels))

accuracy = accuracy_score(all_labels, all_preds)
macro_f1 = f1_score(all_labels, all_preds, average='macro')
print("📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print(f"Best validation F1: {best_val_f1:.4f} (epoch {np.argmax(val_f1_list)+1})")

# رسم نمودارها (کوتاه‌شده برای سادگی)
plt.figure()
plt.plot(val_f1_list, label="F1 (macro)")
plt.plot(val_acc_list, label="Accuracy")
plt.title("Validation Scores per Epoch")
plt.legend()
plt.grid()
plt.savefig(f"{output_dir}f1_curve.png")
plt.close()

Epoch 1: Val Acc=0.4089, Val F1=0.4588
Epoch 2: Val Acc=0.4743, Val F1=0.5458
Epoch 3: Val Acc=0.6052, Val F1=0.6538
Epoch 4: Val Acc=0.5911, Val F1=0.6428
Epoch 5: Val Acc=0.6002, Val F1=0.6323
Epoch 6: Val Acc=0.6777, Val F1=0.6997
Epoch 7: Val Acc=0.6828, Val F1=0.7289
Epoch 8: Val Acc=0.6918, Val F1=0.7320
Epoch 9: Val Acc=0.6898, Val F1=0.7365
Epoch 10: Val Acc=0.7039, Val F1=0.7389
Epoch 11: Val Acc=0.6959, Val F1=0.7433
Epoch 12: Val Acc=0.7130, Val F1=0.7402
Epoch 13: Val Acc=0.7221, Val F1=0.7740
Epoch 14: Val Acc=0.7382, Val F1=0.7791
Epoch 15: Val Acc=0.7261, Val F1=0.7704
Epoch 16: Val Acc=0.7331, Val F1=0.7892
Epoch 17: Val Acc=0.7492, Val F1=0.7969
Epoch 18: Val Acc=0.7462, Val F1=0.7889
Epoch 19: Val Acc=0.7644, Val F1=0.8175
Epoch 20: Val Acc=0.7573, Val F1=0.8046
Epoch 21: Val Acc=0.7633, Val F1=0.8000
Epoch 22: Val Acc=0.7583, Val F1=0.8108
Epoch 23: Val Acc=0.7664, Val F1=0.8175
Epoch 24: Val Acc=0.7674, Val F1=0.8203
Epoch 25: Val Acc=0.7744, Val F1=0.8164
Epoch 26:

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# تنظیمات
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 5  # تعداد کلاس‌ها بر اساس RNALocate (قابل تنظیم)
batch_size = 64
num_epochs = 60
output_dir = "F:\\payan-nameh\\faz2 . 1404.04.02\\Date\\RNALocate\\"

# 🔹 بارگذاری داده‌ها
# -------------------------------
# بارگذاری داده‌های attention (گام ۳)
X_train_attention = np.load(f"{output_dir}X_train_attention.npy")
X_val_attention = np.load(f"{output_dir}X_val_attention.npy")
X_test_attention = np.load(f"{output_dir}X_test_attention.npy")

y_train = np.load(f"{output_dir}y_train.npy")
y_val = np.load(f"{output_dir}y_val.npy")
y_test = np.load(f"{output_dir}y_test.npy")

# -------------------------------
# بارگذاری داده‌های GAN (گام ۴)
X_gan = np.load(f"{output_dir}X_train_augmented.npy")
y_gan = np.load(f"{output_dir}y_train_augmented.npy")

# -------------------------------
# هماهنگ‌سازی ابعاد GAN با شبکه (راه حل شماره 2)
class GanReducer(nn.Module):
    def __init__(self, input_dim=512, output_dim=256):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.linear(x)

gan_reducer = GanReducer(input_dim=X_gan.shape[1], output_dim=X_train_attention.shape[1]).to(device)
X_gan = X_gan.astype(np.float32)
X_gan = torch.FloatTensor(X_gan).to(device)
with torch.no_grad():
    X_gan = gan_reducer(X_gan).cpu().numpy()

# -------------------------------
# ترکیب attention + GAN
X_train_combined = np.concatenate([X_train_attention, X_gan], axis=0)
y_train_combined = np.concatenate([y_train, y_gan], axis=0)

# -------------------------------
# شافل نهایی
X_train_combined, y_train_combined = shuffle(X_train_combined, y_train_combined, random_state=42)

# -------------------------------
# نرمال‌سازی (اختیاری)
input_dim = X_train_combined.shape[1]

# -------------------------------
# ساخت دیتاست و لودر
train_dataset = TensorDataset(torch.FloatTensor(X_train_combined), torch.LongTensor(y_train_combined))
val_dataset = TensorDataset(torch.FloatTensor(X_val_attention), torch.LongTensor(y_val))
test_dataset = TensorDataset(torch.FloatTensor(X_test_attention), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 🔶 گام 5.3: ساخت مدل نهایی (راه حل شماره 4: ساده‌سازی مدل)
class FinalClassifier(nn.Module):
    def __init__(self, input_dim, num_classes=5):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
            # برای تست مدل عمیق‌تر، می‌تونی اینو فعال کنی:
            # nn.Linear(input_dim, 1024),
            # nn.LayerNorm(1024),  # کامنت‌شده (راه حل شماره 3)
            # nn.GELU(),
            # nn.Dropout(0.3),
            # nn.Linear(1024, 512),
            # nn.LayerNorm(512),  # کامنت‌شده (راه حل شماره 3)
            # nn.GELU(),
            # nn.Dropout(0.3),
            # nn.Linear(512, 256),
            # nn.LayerNorm(256),  # کامنت‌شده (راه حل شماره 3)
            # nn.GELU(),
            # nn.Dropout(0.3),
            # nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

model = FinalClassifier(input_dim=input_dim).to(device)

# 🔶 گام 5.4: تعریف loss + optimizer + scheduler (راه حل شماره 1: بازگشت به CrossEntropyLoss)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_combined), y=y_train_combined)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)  # بازگشت به CrossEntropyLoss
# برای تست Focal Loss، می‌تونی اینو فعال کنی:
# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2.0, weight=None):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.weight = weight
#     def forward(self, input, target):
#         ce_loss = nn.CrossEntropyLoss(weight=self.weight, reduction='none')(input, target)
#         pt = torch.exp(-ce_loss)
#         focal_loss = ((1 - pt) ** self.gamma) * ce_loss
#         return focal_loss.mean()
# loss_fn = FocalLoss(gamma=2.0, weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)

# 🔶 گام 5.5: پیاده‌سازی Early Stopping
best_val_f1 = 0.0
patience_counter = 0
val_f1_list, val_acc_list = [], []

for epoch in range(num_epochs):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = loss_fn(out, yb)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_preds.extend(out.argmax(dim=1).cpu().numpy())
            val_labels.extend(yb.cpu().numpy())

    val_f1 = f1_score(val_labels, val_preds, average='macro')
    val_acc = accuracy_score(val_labels, val_preds)
    val_f1_list.append(val_f1)
    val_acc_list.append(val_acc)
    print(f"Epoch {epoch+1}: Val Acc={val_acc:.4f}, Val F1={val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), f"{output_dir}best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= 5:
            print("Early stopping triggered!")
            break

    scheduler.step(1.0 - val_f1)

# 🟩 مرحله 6: ارزیابی و تحلیل نتایج (Week 6)
# 🔶 گام 6.1: بارگذاری بهترین مدل و پیش‌بینی روی Test
model.load_state_dict(torch.load(f"{output_dir}best_model.pth"))
model.eval()

all_preds, all_probs, all_labels = [], [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        all_preds.extend(out.argmax(dim=1).cpu().numpy())
        all_probs.extend(out.cpu().numpy())
        all_labels.extend(yb.cpu().numpy())

# 🔸 2. ذخیره all_preds, all_probs, all_labels
np.save(f"{output_dir}all_preds.npy", np.array(all_preds))
np.save(f"{output_dir}all_probs.npy", np.array(all_probs))
np.save(f"{output_dir}all_labels.npy", np.array(all_labels))

# 🔶 گام 6.2: محاسبه معیارها
accuracy = accuracy_score(all_labels, all_preds)
macro_precision = precision_score(all_labels, all_preds, average='macro')
weighted_precision = precision_score(all_labels, all_preds, average='weighted')
macro_recall = recall_score(all_labels, all_preds, average='macro')
weighted_recall = recall_score(all_labels, all_preds, average='weighted')
macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')
cm = confusion_matrix(all_labels, all_preds)

print("📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

with open(f"{output_dir}classification_report.txt", "w") as f:
    f.write(f"Accuracy: {accuracy:.4f}\n")
    f.write(f"Macro-Precision: {macro_precision:.4f}\n")
    f.write(f"Weighted-Precision: {weighted_precision:.4f}\n")
    f.write(f"Macro-Recall: {macro_recall:.4f}\n")
    f.write(f"Weighted-Recall: {weighted_recall:.4f}\n")
    f.write(f"Macro-F1: {macro_f1:.4f}\n")
    f.write(f"Weighted-F1: {weighted_f1:.4f}\n")

# 🔸 3. نمایش روند یادگیری با ذخیره بهترین epoch
print(f"Best validation F1: {best_val_f1:.4f} (epoch {np.argmax(val_f1_list)+1})")

# ✅ مرحله 2: رسم نمودارهای دقیق برای مقایسه‌ی کلاس‌ها
# گزارش دقیق
report_dict = classification_report(all_labels, all_preds, output_dict=True)

# استخراج مقادیر
classes = [str(i) for i in range(num_classes)]
precision_vals = [report_dict[c]['precision'] for c in classes]
recall_vals = [report_dict[c]['recall'] for c in classes]
f1_vals = [report_dict[c]['f1-score'] for c in classes]
support_vals = [report_dict[c]['support'] for c in classes]

# ساخت DataFrame برای پلات‌ها
df_metrics = pd.DataFrame({
    'Class': classes,
    'Precision': precision_vals,
    'Recall': recall_vals,
    'F1-Score': f1_vals,
    'Support': support_vals
})

# Bar plot for precision, recall, f1
plt.figure(figsize=(12, 6))
df_metrics.set_index('Class')[['Precision', 'Recall', 'F1-Score']].plot(kind='bar', figsize=(12, 6))
plt.title('Per-Class Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1.1)
plt.grid(True)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(f"{output_dir}per_class_metrics_bar.png")
plt.close()

# Plot class support
plt.figure(figsize=(8, 4))
sns.barplot(x='Class', y='Support', data=df_metrics, palette='pastel')
plt.title("Number of Samples per Class (Support)")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{output_dir}class_support.png")
plt.close()

# Normalized confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=range(num_classes))
disp.plot(cmap='Blues', values_format=".2f")
plt.title("Normalized Confusion Matrix")
plt.savefig(f"{output_dir}normalized_confusion_matrix.png", bbox_inches='tight')
plt.close()

# 🔶 گام 6.3: رسم و ذخیره نمودارها (قبلی)
# نمودار F1 و Accuracy در طول زمان
plt.figure()
plt.plot(val_f1_list, label="F1 (macro)")
plt.plot(val_acc_list, label="Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("Validation Scores per Epoch")
plt.legend()
plt.grid()
plt.savefig(f"{output_dir}f1_curve.png", bbox_inches='tight')
plt.close()

# نمودار ROC-AUC
y_bin = label_binarize(all_labels, classes=range(num_classes))
probs = np.array(all_probs)

plt.figure()
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_bin[:, i], probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (One-vs-Rest)")
plt.legend(loc="lower right")
plt.savefig(f"{output_dir}roc_curve.png", bbox_inches='tight')
plt.close()

Epoch 1: Val Acc=0.6647, Val F1=0.6866
Epoch 2: Val Acc=0.6324, Val F1=0.6630
Epoch 3: Val Acc=0.6949, Val F1=0.7192
Epoch 4: Val Acc=0.6999, Val F1=0.7324
Epoch 5: Val Acc=0.7210, Val F1=0.7596
Epoch 6: Val Acc=0.7351, Val F1=0.7581
Epoch 7: Val Acc=0.7482, Val F1=0.7859
Epoch 8: Val Acc=0.7472, Val F1=0.7833
Epoch 9: Val Acc=0.7472, Val F1=0.7891
Epoch 10: Val Acc=0.7633, Val F1=0.8054
Epoch 11: Val Acc=0.7795, Val F1=0.8283
Epoch 12: Val Acc=0.7623, Val F1=0.8078
Epoch 13: Val Acc=0.7644, Val F1=0.8114
Epoch 14: Val Acc=0.7694, Val F1=0.8209
Epoch 15: Val Acc=0.7865, Val F1=0.8315
Epoch 16: Val Acc=0.7986, Val F1=0.8464
Epoch 17: Val Acc=0.8077, Val F1=0.8607
Epoch 18: Val Acc=0.8026, Val F1=0.8467
Epoch 19: Val Acc=0.7895, Val F1=0.8396
Epoch 20: Val Acc=0.7855, Val F1=0.8360
Epoch 21: Val Acc=0.8117, Val F1=0.8592
Epoch 22: Val Acc=0.8006, Val F1=0.8492
Early stopping triggered!
📊 Classification Report:
              precision    recall  f1-score   support

           0     0.8133

<Figure size 1200x600 with 0 Axes>